In [2]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

gender_folder = r"C:\Users\agboo\Downloads\Gender_Classification"
gender_class = ["women","men"]

# we will use this to traverse our features and append them along with their classes
# we will make every picture 100 X 120  this is due to most pictures being selfies / and or full body
row = 120
col = 100
feature = []
def training():
    for gender in range(2):
        path = os.path.join(gender_folder, gender_class[gender])
        # creating flag we will classify with either 0 or 1
        flag = gender_class.index(gender_class[gender]) 
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
                new_img = cv2.resize(img_array,(row,col))
                feature.append([new_img,flag])
            except:
                print(" Error: Corrupted File :( ")
training()
print(len(feature))

# import random so we can shuffle data
import random
random.shuffle(feature)

X = []
Y = []

for feat , ident in feature:
    X.append(feat)
    Y.append(ident)
X = np.array(X).reshape(-1,row,col,3) # features
Y = np.array(Y) # flags


## now we will save the data as pickle files to be loaded into the model
import pickle 
X_out = open("Feat.pickle","wb") # open file for data to then be dumped
pickle.dump(X, X_out)
X_out.close()

Y_out = open("Ident.pickle","wb") # the same is done for the flag list
pickle.dump(Y, Y_out)
Y_out.close()

#### the data will be loaded using rb (read binary when we start to construct the model




    
    

    

    

    
    
    

 Error: Corrupted File :( 
 Error: Corrupted File :( 
 Error: Corrupted File :( 
 Error: Corrupted File :( 
1581


In [29]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , MaxPooling2D , Flatten , Activation
from tensorflow.keras.callbacks import TensorBoard


import time
import pickle

X = pickle.load(open("Feat.pickle","rb"))
Y = pickle.load(open("Ident.pickle","rb"))

X = X/255.0 # this will normalize the feature array

# we will define this to display linked list
def pretty(root):
    while root:
        print(str(root.val) + "->",end="")
        root = root.next
    if root:
        print(root.val)
    else:
        print(None)
# we will have a conv node 
#hidden layers
# then a flatten layer
#dense layers
#output layer

# we will experiment with how we can improve performace by making our dense and fully connected layers bigger , as well as dense layers

 # globally create this model technically sequential but just with more steps not really making use of functional or subclassing like
# we wanted 


model = Sequential() # make model global

# window encompasses maxpoooling convolutions

# IMPORTANT "dimensions" and "nodes" are synoymous
def conv(dimensions , window):
    model.add(Conv2D(dimensions ,(window,window) , input_shape = X.shape[1:]))
    model.add(Activation("relu")) # hardcoding relu within the convolutional layer because we are focused on feature extraction
    model.add(MaxPooling2D(pool_size = (window , window)))

def hidden(dimensions , window):
    model.add(Conv2D(dimensions,(window,window)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size = (window,window)))

# general hidden layers this will be made arbitarily long

def flatter():
    model.add(Flatten())

def dense(dimensions):
    model.add(Dense(dimensions))
    model.add(Activation("sigmoid"))
    model.add(Dense(dimensions))
    model.add(Activation("relu"))
    model.add(Dense(dimensions))
    model.add(Activation("sigmoid"))

# will make this arbitrarily big

def output():
    model.add(Dense(2))
    model.add(Activation("softmax")) # softmax because we have two classes
    model.compile(loss = "binarycrossentropy", optimizer = "adam" ,metrics = ["accuracy"]) 
## creating link classes

class ConvNode():
    def __init__(self,nodes, window, val = 0 , next = None): # non default arguments must be before default unless you call half manually/non default
        self.val = val
        self.next = next
        conv(nodes,window) # we can succesuflly invoke the function this way
class HiddenNode():
    def __init__(self,nodes,window,val = 0, next = None):
        self.val = val
        self.next = next
        hidden(nodes,window) # hidden call

class Flat_Node():
    def __init__(self,val = 0,next = None):
        self.val = val
        self.next = next
        flatter()

class Dense_Node():
    def __init__(self,nodes,val = 0,next = None):
        self.val = val
        self.next = None
        dense(nodes) #dense call
class Output_Node():
    def __init__(self,val = 0, next = None):
        self.val = val
        self.next = None
        output()

def create(nodes , window, hidden_length, dense_length): # dimension , pooling size, length of hidden and length of dense
    count = 1
    cur = ConvNode(nodes,window,count)
    count += 1
    out = cur
    for i in range(hidden_length):
        cur.next = HiddenNode(nodes,window , count)
        cur = cur.next
        count += 1
    cur.next = Flat_Node(count)
    cur = cur.next
    count += 2
    for j in range(dense_length):
        cur.next = Dense_Node(nodes,count)
        cur = cur.next
        count += 1
    cur.next = Output_Node(count)
    cur = cur.next
    count += 1
    # display link
    return pretty(out)
  

create(72,3,2,2)
model.summary() # we will stop here 8/3/24 1:36 AM ; 153,074 Parameters ; Space : 597.95KB

# name model maybe "Face Track"?


        
















        


    
        
        
        
        
   



1->2->3->4->6->7->8->None


Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_92 (Conv2D)                   │ (None, 118, 98, 72)         │           2,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_125 (Activation)          │ (None, 118, 98, 72)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_92 (MaxPooling2D)      │ (None, 39, 32, 72)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_93 (Conv2D)                   │ (None, 37, 30, 72)          │          46,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_126 (Activation)          │ (None, 37, 30, 72)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_93 (MaxPooling2D)      │ (None, 12, 10, 72)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_94 (Conv2D)                   │ (None, 10, 8, 72)           │          46,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_127 (Activation)          │ (None, 10, 8, 72)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_94 (MaxPooling2D)      │ (None, 3, 2, 72)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_20 (Flatten)                 │ (None, 432)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 72)                  │          31,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_128 (Activation)          │ (None, 72)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_129 (Activation)          │ (None, 72)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_130 (Activation)          │ (None, 72)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_131 (Activation)          │ (None, 72)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_132 (Activation)          │ (None, 72)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 153,074 (597.95 KB)

 Trainable params: 153,074 (597.95 KB)

 Non-trainable params: 0 (0.00 B)